## Analyze whether SNOW varies more from year to year or from place to place.

In [1]:
import pandas as pd
import numpy as np
import urllib
import math

In [2]:
import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/computeStats.py'])

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [3]:
import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

In [4]:
### Read the data frame from pickle file

data_dir='../../Data/Weather'
file_index='SSSBSBBS'
meas='SNOW'

from pickle import load

#read statistics
filename=data_dir+'/STAT_%s.pickle'%file_index
STAT,STAT_Descriptions = load(open(filename,'rb'))
print('keys from STAT=',STAT.keys())

#!ls -ld $data_dir/*.parquet

#read data
filename=data_dir+'/decon_%s_%s.parquet'%(file_index,meas)

df=sqlContext.read.parquet(filename)
print(df.count())
df.show(2)

('keys from STAT=', ['TMIN', 'TOBS', 'TMAX', 'SNOW', 'SNWD', 'PRCP'])
940
+------------------+------------------+------------------+-------------------+---------+--------+--------+---------+-----------+------------------+------------------+------------------+------------------+------------------+-----------+---------+------+--------------------+------+
|           coeff_1|           coeff_2|           coeff_3|            coeff_4|elevation|   label|latitude|longitude|measurement|             res_1|             res_2|             res_3|             res_4|          res_mean|    station|total_var|undefs|              vector|  year|
+------------------+------------------+------------------+-------------------+---------+--------+--------+---------+-----------+------------------+------------------+------------------+------------------+------------------+-----------+---------+------+--------------------+------+
| -8.14818275912846|22.498509180840806|41.301338938197496|-24.469225189149892|   15

In [5]:
print df.columns

['coeff_1', 'coeff_2', 'coeff_3', 'coeff_4', 'elevation', 'label', 'latitude', 'longitude', 'measurement', 'res_1', 'res_2', 'res_3', 'res_4', 'res_mean', 'station', 'total_var', 'undefs', 'vector', 'year']


In [6]:
def getPDF(coeff_index):
    #extract longitude and latitude for each station
    sqlContext.registerDataFrameAsTable(df,'weather')
    #Features=', '.join(['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'latitude', 'longitude',\
    #          'res_1', 'res_2', 'res_3', 'res_mean', 'year'])
    Features='station, year, coeff_%d'%coeff_index
    Query="SELECT %s FROM weather"%Features
    print(Query)
    pdf = sqlContext.sql(Query).toPandas()
    return pdf
pdf = getPDF(1)

SELECT station, year, coeff_1 FROM weather


In [7]:
def getYearStationTable(coeff_index):
    pdf = getPDF(coeff_index)
    year_station_table=pdf.pivot(index='year', columns='station', values='coeff_%d'%coeff_index)
    return year_station_table
year_station_table = getYearStationTable(1)
year_station_table.head(10)

SELECT station, year, coeff_1 FROM weather


station,US1CAED0007,US1CAED0009,US1NVDG0002,US1NVLY0003,USC00040943,USC00041072,USC00041075,USC00041277,USC00041909,USC00041911,...,USC00267620,USC00267640,USC00267806,USC00268034,USC00268186,USC00268202,USC00268822,USC00268977,USC00269137,USC00269229
year,,,,,,,,,,,,,,,,,,,,,
1904.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1908.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1909.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1910.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1911.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1912.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1913.0,NaN,NaN,NaN,NaN,NaN,35.386744,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1914.0,NaN,NaN,NaN,NaN,NaN,45.982394,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1915.0,NaN,NaN,NaN,NaN,NaN,66.970913,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-26.47667


### Estimating the effect of the year vs the effect of the station

To estimate the effect of time vs. location on the first eigenvector coefficient we
compute:

* The average row: `mean-by-station`
* The average column: `mean-by-year`

We then compute the RMS before and after subtracting either  the row or the column vector.

In [8]:
def estimateEffect(coeff_index):
    year_station_table = getYearStationTable(coeff_index)
    def RMS(Mat):
        return np.sqrt(np.nanmean(Mat**2))

    def MS(Mat):
        return np.nanmean(Mat**2)
    
    mean_by_year=np.nanmean(year_station_table,axis=1)
    mean_by_station=np.nanmean(year_station_table,axis=0)
    
    tbl_minus_year = year_station_table
    tbl_minus_station = year_station_table
    #for i in range(coeff_index):
    #    print "Minus %d"%i
    tbl_minus_year = (tbl_minus_year.transpose()-mean_by_year).transpose()
    tbl_minus_station = tbl_minus_station-mean_by_station

    print 'total RMS                   = %.2f'%RMS(year_station_table)
    print 'RMS removing mean-by-station= %.2f'%RMS(tbl_minus_station)
    print 'RMS removing mean-by-year   = %.2f'%RMS(tbl_minus_year)
    print '----------------------------------------------------'
    print 'total MS                   = %.2f'%MS(year_station_table)
    ms0 = MS(year_station_table)
    print 'MS removing mean-by-station= %.2f'%MS(tbl_minus_station)
    ms1 = MS(tbl_minus_station)
    print 'fraction explained = %.1f'%(100.-100.*ms1/ms0)
    print 'MS removing mean-by-year   = %.2f'%MS(tbl_minus_year)
    ms2 = MS(tbl_minus_year)
    print 'fraction explained = %.1f'%(100.-100.*ms2/ms0)
    print '----------------------------------------------------'


In [9]:
top_num_coeff = 4
for i in range(top_num_coeff):
    estimateEffect(i+1)

SELECT station, year, coeff_1 FROM weather
total RMS                   = 78.04
RMS removing mean-by-station= 40.67
RMS removing mean-by-year   = 63.44
----------------------------------------------------
total MS                   = 6090.15
MS removing mean-by-station= 1654.34
fraction explained = 72.8
MS removing mean-by-year   = 4025.21
fraction explained = 33.9
----------------------------------------------------
SELECT station, year, coeff_2 FROM weather
total RMS                   = 51.08
RMS removing mean-by-station= 49.68
RMS removing mean-by-year   = 31.34
----------------------------------------------------
total MS                   = 2609.36
MS removing mean-by-station= 2468.36
fraction explained = 5.4
MS removing mean-by-year   = 982.12
fraction explained = 62.4
----------------------------------------------------
SELECT station, year, coeff_3 FROM weather
total RMS                   = 49.57
RMS removing mean-by-station= 47.97
RMS removing mean-by-year   = 32.22
-----------